In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "VANNVISAL/gemma-3-4b-khmer",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/59.7M [00:00<?, ?B/s]

In [3]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
# def formatting_prompts_func(examples):
#     instructions = examples["input"]
#     outputs      = examples["output"]
#     texts = []
#     for instruction, output in zip(instructions, outputs):
#       messages = [
#         {
#             "role": "user", 'content': instruction.strip(" ")
#           },
#           {
#             "role": "assistant", 'content': output
#           }
#       ]
#       #tokenizer.apply_chat_template(messages, add_generation_prompt=False , tokenize= False)
#       text = tokenizer.apply_chat_template(messages, add_generation_prompt=False , tokenize= False)
#       texts.append(text)
#     return {"text":texts}



In [4]:
from unsloth.chat_templates import get_chat_template

# Set the tokenizer chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="gemma-3",
)

# Define your messages
messages = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": "ទៅមុខ១០មហើយបត់ឆ្វេង",
    }]
}]

# Apply the chat template
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
)

# Tokenize input
inputs = tokenizer([text], return_tensors="pt").to("cuda")

# Generate model output
generated_ids = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=1.0,
    top_p=0.95,
    top_k=64,
)

# Trim the input tokens from output
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

# Decode output
output_text = tokenizer.batch_decode(
    generated_ids_trimmed,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)
# Print result
print(type(output_text))

<class 'list'>


In [5]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "រ៉ូបូតទៅមុខចំងាយបួនម៉ែត្រ and turn left",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{'action':'move','params':{'distance':4,'is_forward':True,'unit':'meter'}}\n{'action':'rotate','params':{'angle':90,'is_clockwise':False,'unit':'degree'}}


In [20]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "រ៉ូបូតទៅមុខចំងាយបួនម៉ែត្រ​រួចហើយរ៉ូបូតទៅមុខចំងាយ6ម៉ែត្របន្ទាប់មករ៉ូបូតទៅមុខចំងាយ5ម៉ែត្រនឹងរ៉ូបូតទៅមុខចំងាយ7ម៉ែត្រ",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{'action':'move','params':{'distance':4,'is_forward':True,'unit':'meter'}}\n{'action':'move','params':{'distance':6,'is_forward':True,'unit':'meter'}}\n{'action':'move','params':{'distance':5,'is_forward':True,'unit':'meter'}}\n{'action':'move','params':{'distance':7,'is_forward':True,'unit':'meter'}}


In [21]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "រ៉ូបូតទៅមុខចំងាយ៤ម៉ែត្រ​រួចហើយរ៉ូបូតទៅមុខចំងាយ6ម៉ែត្របន្ទាប់មករ៉ូបូតទៅមុខចំងាយ5ម៉ែត្រនឹងរ៉ូបូតទៅមុខចំងាយ7ម៉ែត្រហើយនឹងត្រឡប់មកវិញ",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{'action':'move','params':{'distance':4,'is_forward':True,'unit':'meter'}}\n{'action':'move','params':{'distance':6,'is_forward':True,'unit':'meter'}}\n{'action':'move','params':{'distance':5,'is_forward':True,'unit':'meter'}}\n{'action':'move','params':{'distance':7,'is_forward':True,'unit':'meter'}}\n{'action':'move','params':{'distance':4,'is_forward':False,'unit':'meter'}}


In [22]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "មកវិញ៤៦៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥៥ម",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{'action':'move','params':{'distance':465555555555555555555555555555555555555555555555555555,'is_forward':True,'unit':'meter'}}


In [29]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "រ៉ូបូតទៅមុខចំងាយ5ម៉ែត្រនឹងរ៉ូបូតទៅមុខទៀតទៅហើយនឹងត្រឡប់មកវិញ",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{'action':'move','params':{'distance':5,'is_forward':True,'unit':'meter'}}\n{'action':'move','params':{'distance':5,'is_forward':True,'unit':'meter'}}\n{'action':'move','params':{'distance':10,'is_forward':False,'unit':'meter'}}


In [28]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "រ៉ូបូតទៅមុខចំងាយ5ម៉ែត្រ",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)
output = "".join(output_text)
test = output.split("\\n")

{'action':'move','params':{'distance':5,'is_forward':True,'unit':'meter'}}


In [26]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "ទៅមុខទៀតទៅហើយនឹងត្រឡប់មកវិញ",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{{''}}


In [33]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "ទៅមុខទៀតទៅ៤មហើយនឹងបត់ឆ្វេង",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{'action':'move','params':{'distance':4.0,'is_forward':True,'unit':'meter'}}\n{'action':'rotate','params':{'angle':90.0,'is_clockwise':False,'unit':'degree'}}


In [34]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "បត់ឆ្វេង​បត់ឆ្វេងបត់ឆ្វេងបត់ឆ្វេងបត់ឆ្វេង",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{'action':'rotate','params':{'angle':90,'is_clockwise':False,'unit':'degree'}}


In [41]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "បត់ឆ្វេង​​​ហើយទៅមុខ៦ម៉ែត្រ",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{'action':'rotate','params':{'angle':90,'is_clockwise':False,'unit':'degree'}}\n{'action':'move','params':{'linear_speed':2.1,'distance':6.0,'is_forward':True,'unit':'meter'}}


In [42]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "ទៅមុខ៦ម៉ែត្រ​​​ហើយបត់ស្ដាំ",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True,skip_special_tokens=True,clean_up_tokenization_spaces=False),
)

{'action':'move','params':{'distance':6,'is_forward':True,'unit':'meter'}}\n{'action':'rotate','params':{'angle':90,'is_clockwise':True,'unit':'degree'}}
